In [1]:
import tensorflow as tf
import os 
import numpy as np
from keras.utils import image_utils

from siameFC.SiameFC import siameFc_model
from siameFC.loss_of_scoreMap import loss_of_scoreMap
from siameFC.utils import  load_images, make_ground_th_label

tf.compat.v1.disable_eager_execution()

In [2]:
Z_SHAPE = (127, 127, 3)
X_SHAPE = (255, 255, 3)

In [3]:
#data path 
x_dir = "./sample/VOT19/car1/x/" #number of 742
z_dir = "./sample/VOT19/car1/z/" #number of 1

In [4]:
#img name list
x_name_lsit = os.listdir(x_dir)
z_name_lsit = os.listdir(z_dir)

In [5]:
#original image size
org_img = image_utils.load_img(x_dir + x_name_lsit[0])
org_img_tensor = image_utils.array_to_img(org_img)
org_img_size = org_img_tensor.size #VOT img size : 640x480

In [6]:
#make image list
data_size = len(os.listdir(x_dir)) #VOT car1 data_size : 742

x_images = load_images(x_dir, x_name_lsit, 255, data_size, input='x')
z_images = load_images(z_dir, z_name_lsit, 127, data_size, input='z')

#data normalization
x_images = x_images / 255.
z_images = z_images / 255.

In [7]:
#ground truth
ground_th_dir = "./sample/VOT19/car1/label/groundtruth.txt"
ground_th = np.loadtxt(ground_th_dir, delimiter=',') #shape (742,8)


response_size = 17 #score map size
final_stride = 8 
label = make_ground_th_label(data_size, final_stride, response_size, ground_th, org_img_size)


In [8]:

model = siameFc_model(X_SHAPE,Z_SHAPE)



opt = tf.keras.optimizers.legacy.SGD( 
         name='SGD', learning_rate= 0.0001
        )


        
model.compile(optimizer=opt, loss=loss_of_scoreMap, metrics=['accuracy'])

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use fn_output_signature instead
Tensor("loss/lambda_loss/Mean:0", shape=(None,), dtype=float32)


In [9]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 255, 255, 3  0           []                               
                                )]                                                                
                                                                                                  
 input_1 (InputLayer)           [(None, 127, 127, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_5 (Conv2D)              (None, 123, 123, 96  34944       ['input_2[0][0]']                
                                )                                                             

In [10]:
batch_size = 8 
epochs = 50 


model.fit([x_images, z_images], [label], batch_size=batch_size, epochs=epochs)

Train on 742 samples
Epoch 1/50
 32/742 [>.............................] - ETA: 4:31 - loss: nan - accuracy: 0.6103 

KeyboardInterrupt: 